In [ ]:
%pip install mne-icalabel

In [1]:
# Imports
import os
from ipywidgets import *
import numpy as np
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import utils

# Specify graph rendering method
# %matplotlib widget
plt.switch_backend("TkAgg")

In [2]:
DATASET_PATH = "./dataset"
FILENAME_TEMPLATE = "TMS-EEG-H_02_S1b_{}_{}.vhdr"

spTEP_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "pre")), preload=True
)
sampling_rate = spTEP_pre_raw.info["sfreq"]
events, event_dict = mne.events_from_annotations(spTEP_pre_raw)
tms_indices = [event[0] for event in events if event[2] == 1]

rsEEG_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "pre")), preload=True
)

Extracting parameters from ./dataset/TMS-EEG-H_02_S1b_spTEP_pre.vhdr...
Setting channel info structure...
Reading 0 ... 2696199  =      0.000 ...   539.240 secs...
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Extracting parameters from ./dataset/TMS-EEG-H_02_S1b_rsEEG_pre.vhdr...
Setting channel info structure...
Reading 0 ... 3984899  =      0.000 ...   796.980 secs...


In [3]:
# Plotting utilities
def plot_single_response(eeg_data, channel="Pz", tmin=-0.005, tmax=0.01):
    events, event_dict = mne.events_from_annotations(eeg_data)
    event_id = event_dict["Stimulus/S  1"]
    epochs = mne.Epochs(
        eeg_data,
        events,
        event_id=event_id,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        preload=True,
        picks=channel,
    )

    epochs.plot(picks=channel, n_epochs=1, show=True, scalings={"eeg": 50e-4})


def plot_average_epoch(epochs, start=-0.05, end=0.25):
    data = epochs.get_data()
    mean_responses = np.mean(data, axis=0)
    time_points = np.linspace(-1, 1, data.shape[2])
    selected_indices = np.where((time_points >= start) & (time_points <= end))
    for i, mean_response in enumerate(mean_responses):
        selected_data = mean_response[selected_indices]
        selected_time_points = time_points[selected_indices]
        plt.plot(selected_time_points, selected_data, label=f"Channel {i+1}")
    plt.xlabel("Time points")
    plt.ylabel("Mean response")
    plt.show()


def plot_response(eeg):
    utils.plot_average_response(eeg, tmin=-0.05, tmax=0.25)  # Check full response
    utils.plot_single_response(
        eeg, channel="Pz", tmin=-0.05, tmax=0.05
    )  # Check for TMS pulse

# Cleaning - spTEP

The paper of Bertazzoli et al. (2021) compares 4 pipelines: ARTIST, TMSEEG, TESA and SOUND-SSP-SIR, all of which work decently well in varying degrees. There are common steps, but TESA will be the one that will be most closely followed. The current steps are as follows:

1. Remove EOG
2. Remove TMS pulse
3. Downsample
4. **ICA - 1**
5. Bandpass - Notch filters
6. **ICA - 2**
7. Rereference

Currently, there is no demeaning or bad channel rejection present as in TESA. Demeaning is done before the TMS-pulse interpolation, and baseline correction should be done as last step after rereferencing.


In [ ]:
plot_response(spTEP_pre_raw)

![](img/raw-average.png)
![](img/single-pulse-raw.JPG)

The figure above already shows 2 major artifacts that cleaning will have the biggest impact on: TMS-pulse interpolation and demeaning/baseline correction. The major TMS-pulse artifact falls closely within the range of 2ms before until 5ms after the pulse.


In [ ]:
spTEP_copy = spTEP_pre_raw.copy()

## EOG removal


In [4]:
def remove_EOG(eeg_data):
    eeg_data.drop_channels(["HEOG", "VEOG"])

In [ ]:
remove_EOG(spTEP_copy)

## TMS pulse removal


In [ ]:
def calculate_range_indices(tms_index, start, end, sampling_rate):
    """
    start and end are positive in seconds
    sampling rate in Hz
    """
    samples_before = int(start * sampling_rate)
    samples_after = int(end * sampling_rate)

    start_index = max(0, tms_index - samples_before)
    end_index = tms_index + samples_after

    return start_index, end_index

In [ ]:
def interpolate_TMS_pulse(eeg_data_raw, tms_indices, start, end, sampling_rate):
    eeg_data = eeg_data_raw.get_data()
    num_electrodes = eeg_data.shape[0]
    for tms_index in tms_indices:
        start_index, end_index = calculate_range_indices(
            tms_index, start, end, sampling_rate
        )
        for i in range(num_electrodes):
            x = [start_index - 2, start_index - 1, end_index + 1, end_index + 2]
            y = [
                eeg_data[i, start_index - 2],
                eeg_data[i, start_index - 1],
                eeg_data[i, end_index + 1],
                eeg_data[i, end_index + 2],
            ]
            x_new = np.arange(start_index, end_index + 1)

            interp_func = interp1d(x, y, kind="cubic")
            eeg_data[i, start_index : end_index + 1] = interp_func(x_new)

    eeg_data_raw._data = eeg_data

In [ ]:
interpolate_TMS_pulse(
    spTEP_copy, tms_indices, 0.005, 0.01, sampling_rate
)  # 2ms before, 5ms after

In [ ]:
plot_response(spTEP_copy)

![](img/interpolation-5-10/interpolated-average.png)
![](img/interpolation-5-10/single-pulse-interpolated.JPG)


This succesfully removed the TMS pulse artifact by using cubic interpolation based on the 2 values before and 2 values after the range that is to be interpolated.


## Downsampling

The original data was captured with a sampling frequency of 5000 Hz. 1000 Hz is chosen as the frequency to be downsampled to, as this means that, following Nyquists theorem, the highest frequency that will be accurately recorded is 500 Hz, which should be more than enough for further analysis, as the gamma band is often referred to as 30-100 Hz.


In [5]:
def downsample(eeg_data, sample_rate=1000):
    eeg_data.resample(sample_rate, npad="auto")

In [ ]:
downsample(spTEP_copy)

In [ ]:
plot_response(spTEP_copy)

![](img/interpolation-5-10/single-downsampled.JPG)


## Epoching


In [ ]:
def epoching(eeg_data):
    events, event_dict = mne.events_from_annotations(eeg_data)
    event_id = event_dict["Stimulus/S  1"]
    epochs = mne.Epochs(
        eeg_data,
        events,
        event_id=event_id,
        tmin=-1,
        tmax=1,
        baseline=None,
        preload=True,
    )
    return epochs

In [ ]:
epochs = epoching(spTEP_copy)

In [ ]:
plot_average_epoch(epochs)

![](img/interpolation-5-10/average_epoch.png)


## Demeaning/detrending

Demeaning is achieved by subtracting each value from each electrode with the average value of the corresponding electrode, essentially bringing the means from all electrodes to 0.

> TODO: check if other way of demeaning on complete electrode is possible to move value near 0 or better yet on 0


In [ ]:
def demean(eeg_data):
    eeg_data.apply_function(lambda x: x - x.mean())

In [ ]:
demean(spTEP_copy)

In [ ]:
data = spTEP_copy.get_data()
mean_values = np.mean(data, axis=1)
total_mean = np.mean(mean_values)
print(f"Total mean: {total_mean}")

plot_response(spTEP_copy)

In [ ]:
def demean_epochs(epochs):
    data = epochs.get_data()
    demeaned_data = data - np.mean(data, axis=2, keepdims=True)
    demeaned_epochs = mne.EpochsArray(
        demeaned_data, epochs.info, events=epochs.events, event_id=epochs.event_id
    )
    return demeaned_epochs

In [ ]:
epochs = demean_epochs(epochs)

In [ ]:
plot_average_epoch(epochs)

Below are two graphs with demeaning applied to both the full electrode and to individual epochs. Clearly, there has to be some error in demeaning the full electrode, as those averages aren't close enough to 0. However, when printing the mean values of the electrodes they are indeed close to 0 (e.g. 1\*e-20). For now, as the demeaned epochs are the desired result, the epochs will be used for further cleaning.

![](img/interpolation-5-10/average-demean.png)
![](img/interpolation-5-10/average_epoch_demeaned.png)


## ICA - 1

The first ICA filter is mainly to remove the primary large artifacts such as muscle and electrical charge. If demeaning were applied now, a graph as below is the result.

This is implemented by first fitting ICA to the signal, and then applying the threshold formula used by the TESA software to each component to either keep or remove each ICA component.


In [ ]:
def ICA_1(epoch_data, T=3.5, b1=0.011, b2=0.030, n_components=20):
    ica = ICA(n_components=n_components, random_state=97)
    ica.fit(epoch_data)

    # Credits to Arne Callaert for the following code
    sources = ica.get_sources(epoch_data)
    averaged_sources = sources.get_data().mean(axis=0)
    times = sources.times
    sfreq = sources.info["sfreq"]
    indices = np.where((times >= (b1 / 1000)) & (times <= (b2 / 1000)))
    print("indices:", indices)
    components_to_remove = []

    for i, component in enumerate(averaged_sources):
        base = len(times) / 2
        b1_index = int(base + (b1 * sfreq))
        b2_index = int(base + (b2 * sfreq))
        x = np.mean(np.abs(component[b1_index:b2_index]))
        y = np.mean(np.abs(component))
        if x / y > T:
            print("FOUND:", x / y)
            components_to_remove.append(i)

    ica.exclude = components_to_remove

    epoch_data = ica.apply(epoch_data)

In [ ]:
ICA_1(epochs)

In [ ]:
plot_average_epoch(epochs)

![](img/interpolation-5-10/average_epoch_ica_1.png)
The graph below shows the result after applying the ICA filter. However, this looks like it results in muddier values than before. The goal was to filter out the initial peak values, residue from the TMS pulse, but these are still present.


## Bandpass - Notch


In [ ]:
def bandpass_notch(epoch_data, low_freq=1, high_freq=100, notch_freqs=[50]):
    # Bandpass
    epoch_data.filter(low_freq, high_freq)

    # Notch (only directly available on raw object, not on epochs)
    data = epoch_data.get_data()
    notch_filtered = mne.filter.notch_filter(data, epochs.info["sfreq"], notch_freqs)
    filtered_epochs = mne.epochs.EpochsArray(
        notch_filtered, epochs.info, events=epochs.events, tmin=epochs.tmin
    )

    return filtered_epochs

In [ ]:
epochs = bandpass_notch(epochs)

In [ ]:
plot_average_epoch(epochs)

![](img/interpolation-5-10/average_epoch_filter.png)


## Rereference


In [ ]:
def rereference(epochs):
    mne.set_eeg_reference(epochs, ref_channels="average")

In [ ]:
rereference(epochs)

In [ ]:
plot_average_epoch(epochs)

![](img/interpolation-5-10/average_epoch_rereference.png)


## ICA - 2

Li et al., (2022). MNE-ICALabel: Automatically annotating ICA components with ICLabel in Python. Journal of Open Source Software, 7(76), 4484, https://doi.org/10.21105/joss.04484


In [ ]:
def ICA_2(epoch_data):
    ica = mne.preprocessing.ICA(n_components=20, random_state=42)
    ica.fit(epoch_data)
    ic_labels = label_components(epoch_data, ica, method="iclabel")

    print(ic_labels["labels"])

    labels = ic_labels["labels"]
    exclude_idx = [
        idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
    ]
    print(f"Excluding these {len(exclude_idx)} ICA components: {exclude_idx}")

    ica.apply(epoch_data, exclude=exclude_idx)

In [ ]:
epochs_copy = epochs.copy()

In [ ]:
ICA_2(epochs_copy)

In [ ]:
plot_average_epoch(epochs_copy)

![ica 2 result](img/interpolation-5-10/average_epoch_ica_2.png)


## Baseline correction


In [ ]:
def baseline(epoch_data):
    epoch_data.apply_baseline((None, None))

In [ ]:
baseline(epochs_copy)

In [ ]:
plot_average_epoch(epochs_copy)

![](img/interpolation-5-10/average_epoch_baseline.png)


In [ ]:
spTEP_pre_copy = spTEP_pre_raw.copy()
remove_EOG(spTEP_pre_copy)

spTEP_pre_copy.compute_psd().plot_topomap()

In [ ]:
spTEP_pre_raw.plot_sensors(show_names=True)

In [ ]:
def plot_full_average_epoch(epochs, electrodes=None, start=-0.05, end=0.25):
    epochs = epochs.copy()
    if electrodes is not None:
        epochs.pick_channels(electrodes)
    data = epochs.get_data()
    mean_responses = np.mean(data, axis=(0, 1))
    sem_responses = np.std(data, axis=(0, 1)) / np.sqrt(data.shape[0])
    time_points = np.linspace(-1, 1, data.shape[2])
    selected_indices = np.where((time_points >= start) & (time_points <= end))
    selected_data = mean_responses[selected_indices]
    selected_sem = sem_responses[selected_indices]
    selected_time_points = time_points[selected_indices]
    plt.plot(selected_time_points, selected_data, label="Average of all electrodes")
    plt.fill_between(
        selected_time_points,
        selected_data - selected_sem,
        selected_data + selected_sem,
        color="b",
        alpha=0.2,
    )
    plt.xlabel("Time points")
    plt.ylabel("Mean response")
    plt.legend()
    plt.show()

## Final result

In these final plots, the total average is plotted with the error range.


In [ ]:
# F, FC
# 3, 5, 7

plot_full_average_epoch(epochs_copy, end=0.3)
plot_full_average_epoch(epochs_copy, ['F3', 'FC5', 'FC1'], end=0.3)

![](img/interpolation-5-10/average_epoch_all_electrodes.png)
![](img/interpolation-5-10/final-local.png)

In [ ]:
# Save the epochs to a file
epochs_copy.save("processed_epochs-epo.fif")

## TODO

Current biggest things to find out:

- is there a way to further improve the filtering that ICA 1 is supposed to achieve? (Filtering out the residue of the TMS pulse)
- how can time ranges be plot on the scalp topography? like in the comparative paper

# Cleaning - rsEEG

Cleaning regular EEG data has similar steps to spTEP data, as it uses the same capturing techniques, but doesn't have to deal with TMS related artifacts and doesn't have the event data from TMS pulses. The most important filtering method used here is ICA filtering using the `mne_icalabel` library, paired with bandpass filtering, notch filtering and rereferencing.


In [6]:
def calculate_avg_std(eeg_data_raw):
    # Get the data
    data = eeg_data_raw.get_data()

    # Calculate the standard deviation
    std = np.std(data, axis=1)

    # Calculate the average standard deviation
    avg_std = np.mean(std)

    return avg_std

In [7]:
def plot_raw_time_domain(eeg_data_raw, start=0, duration=1, channels=None):
    # Get the data and times
    eeg_copy = eeg_data_raw.copy()
    if channels is not None:
        eeg_copy.pick(channels)
    data, times = eeg_copy[:, int(start*eeg_copy.info['sfreq']):int((start+duration)*eeg_copy.info['sfreq'])]

    # Create a new figure
    plt.figure()

    # Plot each channel
    for i in range(data.shape[0]):
        plt.plot(times, data[i, :], label=eeg_copy.info['ch_names'][i])

    # Add y label
    plt.ylabel('Voltage (V)')

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()

In [8]:
rsEEG_copy = rsEEG_pre_raw.copy()

In [17]:
print("{:.6e}".format(calculate_avg_std(rsEEG_pre_raw)))
plot_raw_time_domain(rsEEG_pre_raw)

4.140313e-04


![raw rsEEG](img/rsEEG/rsEEG_raw.png)

## Remove EOG


## Downsampling


## Demean


In [9]:
def demean_brainvis(eeg_data_raw):
    eeg_data_raw.apply_function(lambda x: x - np.mean(x))

## Bandpass & notch


In [10]:
def rsEEG_filters(eeg_data, l_freq=1, h_freq=100, notch_freqs=[50]):
    eeg_data.filter(l_freq, h_freq)
    eeg_data.notch_filter(notch_freqs)

## ICA


In [11]:
def rsEEG_ICA(eeg_data):
    ica = ICA(n_components=20, random_state=97)
    ica.fit(eeg_data)
    ic_labels = label_components(eeg_data, ica, method="iclabel")
    
    labels = ic_labels["labels"]
    exclude_idx = [
        idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
    ]
    
    print(f"Excluding ICA components: {exclude_idx}")
    
    ica.apply(eeg_data, exclude=exclude_idx)

## Rereference


In [12]:
def rsEEG_rereference(eeg_data):
    eeg_data.set_eeg_reference(ref_channels="average")

## Epoching

In [13]:
%pip install -U autoreject


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from autoreject import AutoReject

def rsEEG_epoch(eeg_data, duration=2.0, overlap=0.5):
    """Create an epochs object from the raw brainvision data, where each epoch is 2 seconds long and got baseline corrected using the last 500ms of the previous epoch

    Args:
        eeg_data (Any): full EEG data
    """
    
    overlap *= duration
    
    events = mne.make_fixed_length_events(eeg_data, duration=duration+overlap, overlap=overlap)
    epochs = mne.Epochs(eeg_data, events, baseline=(None, None), tmin=0, tmax=duration, preload=True)
    ar = AutoReject()
    epochs_clean = ar.fit_transform(epochs)
    
    return epochs_clean

## Final

DISCOVER-EEG:
1. Line noise removal
2. Bad channel rejection
3. Rereference
4. ICA
5. Bad channel interpolation
6. Bad time segments removal

Own pipeline without TMS filtering:
1. Remove EOG
2. Downsample
3. Demeaning
4. Bandpass & notch
5. ICA
6. Rereference
7. Epoch & baseline correction

In [15]:
rsEEG_copy = rsEEG_pre_raw.copy()

remove_EOG(rsEEG_copy)
downsample(rsEEG_copy)
demean_brainvis(rsEEG_copy)
rsEEG_filters(rsEEG_copy)
rsEEG_ICA(rsEEG_copy)
rsEEG_rereference(rsEEG_copy)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 23.0s.


/tmp/ipykernel_3500229/1449805445.py:4: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(eeg_data, ica, method="iclabel")
/tmp/ipykernel_3500229/1449805445.py:4: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the extended infomax algorithm, use the 'mne.preprocessing.ICA' instance with the arguments 'ICA(method='infomax', fit_params=dict(extended=True))' (scikit-learn) or 'ICA(method='picard', fit_params=dict(ortho=False, extended=True))' (python-picard).
  ic_labels = label_components(eeg_data, ica, method="iclabel")


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Excluding ICA components: [0, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18]
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 12 ICA components
    Projecting back using 62 PCA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [16]:
epochs = rsEEG_epoch(rsEEG_copy)

Not setting metadata
397 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 397 events and 2001 original time points ...
0 bad epochs dropped
Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/62 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/62 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/397 [00:00<?,       ?it/s]

  0%|          | n_interp : 0/3 [00:00<?,       ?it/s]

...

  0%|          | Repairing epochs : 0/397 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.40 and n_interpolate=4


  0%|          | Repairing epochs : 0/397 [00:00<?,       ?it/s]

Dropped 84 epochs: 1, 2, 3, 4, 5, 7, 8, 9, 11, 14, 16, 17, 18, 19, 20, 21, 55, 79, 80, 81, 90, 106, 107, 108, 118, 119, 122, 130, 134, 146, 148, 155, 160, 165, 166, 167, 168, 183, 189, 218, 228, 234, 244, 245, 259, 263, 279, 285, 286, 287, 288, 302, 306, 310, 311, 313, 314, 315, 319, 320, 321, 322, 323, 324, 325, 326, 336, 340, 347, 348, 350, 356, 358, 359, 368, 369, 370, 373, 374, 375, 388, 389, 395, 396


In [26]:
def save_epochs(epochs, filename):
    epochs.save(filename, overwrite=True)

save_epochs(epochs, 'filtered/rsEEG-epochs-cleaned.fif')

/tmp/ipykernel_3500229/3507919554.py:2: RuntimeWarning: This filename (filtered/rsEEG-epochs-cleaned.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(filename, overwrite=True)


In [ ]:
def load_epochs(filename):
    return mne.read_epochs(filename, preload=True)

epochs = load_epochs('filtered/rsEEG-epochs-cleaned.fif')

In [27]:
def plot_average_response(epochs):
    # Get the average of all epochs
    evoked = epochs.average()

    # Plot the average response
    evoked.plot(time_unit='s')

plot_average_response(epochs)

In [28]:
import numpy as np
import matplotlib.pyplot as plt

def plot_gmfa(epochs):
    # Get the data from the epochs
    data = epochs.get_data()

    # Calculate the GMFA
    gmfa = np.sqrt(np.mean(data ** 2, axis=1))

    # Average the GMFA across all epochs
    avg_gmfa = np.mean(gmfa, axis=0)

    # Plot the average GMFA
    plt.figure()
    plt.plot(epochs.times, avg_gmfa)
    plt.xlabel('Time (s)')
    plt.ylabel('GMFA')
    plt.show()

plot_gmfa(epochs)

def plot_total_average(epochs):
    # Average all epochs into one epoch
    evoked = epochs.average()

    # Get the data from the evoked object
    data = evoked.data

    # Take the average across all channels
    total_average = np.mean(data, axis=0)

    # Plot the total average
    plt.figure()
    plt.plot(evoked.times, total_average)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title('Total Average Response')
    plt.show()

plot_total_average(epochs)

/tmp/ipykernel_3500229/2068037934.py:6: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


In [42]:
def rsEEG_cleaning(eeg_data):
    remove_EOG(eeg_data)
    downsample(eeg_data)
    demean_brainvis(eeg_data)
    rsEEG_filters(eeg_data)
    rsEEG_ICA(eeg_data)
    rsEEG_rereference(eeg_data)
    epochs = rsEEG_epoch(eeg_data)
    return epochs

In [32]:
def plot_average_epoch(epochs, start=-0.05, end=0.25, electrodes=None):
    epochs = epochs.copy()
    if electrodes is not None:
        epochs.pick_channels(electrodes)
    data = epochs.get_data(copy=False)
    mean_responses = np.mean(data, axis=0)
    time_points = np.linspace(-1, 1, data.shape[2])
    selected_indices = np.where((time_points >= start) & (time_points <= end))
    for i, mean_response in enumerate(mean_responses):
        selected_data = mean_response[selected_indices]
        selected_time_points = time_points[selected_indices]
        plt.plot(selected_time_points, selected_data, label=f"Channel {i+1}")
    plt.xlabel("Time points")
    plt.ylabel("Mean response")
    plt.show()
    
def plot_aggregated_average_epoch(epochs, electrodes=None, start=-0.05, end=0.25, plot_gmfp=True):
    epochs = epochs.copy()
    if electrodes is not None:
        epochs.pick_channels(electrodes)
    data = epochs.get_data(copy=False)
    mean_responses = np.mean(data, axis=(0, 1))
    sem_responses = np.std(data, axis=(0, 1)) / np.sqrt(data.shape[0])
    time_points = np.linspace(-1, 1, data.shape[2])
    selected_indices = np.where((time_points >= start) & (time_points <= end))
    selected_data = mean_responses[selected_indices]
    selected_sem = sem_responses[selected_indices]
    selected_time_points = time_points[selected_indices]
    plt.plot(selected_time_points, selected_data, label="Average of all electrodes")
    plt.fill_between(
        selected_time_points,
        selected_data - selected_sem,
        selected_data + selected_sem,
        color="b",
        alpha=0.2,
    )
    if plot_gmfp:
        data = epochs.get_data()
        average_epoch = np.mean(data, axis=0)
        # mean = np.mean(average_epoch, axis=0, keepdims=True)
        # diff = average_epoch - mean
        # squared_diff = np.square(diff)
        # sum_squared_diff = np.sum(squared_diff, axis=0)
        # gmfp = np.sqrt(sum_squared_diff / average_epoch.shape[0])
        
        # Calculate the GMFA by taking the standard deviation accross all channels
        gmfa = np.std(average_epoch, axis=0)
    
        # plt.plot(selected_time_points, gmfp[selected_indices], label="GMFP")
        plt.plot(selected_time_points, gmfa[selected_indices], label="GMFA")
    plt.xlabel("Time points")
    plt.ylabel("Mean response")
    plt.legend()
    plt.show()

In [45]:
rsEEG_copy = rsEEG_pre_raw.copy()
epochs = rsEEG_cleaning(rsEEG_copy)
plot_average_epoch(epochs)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 27.1s.


/tmp/ipykernel_3205077/1449805445.py:4: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(eeg_data, ica, method="iclabel")
/tmp/ipykernel_3205077/1449805445.py:4: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the extended infomax algorithm, use the 'mne.preprocessing.ICA' instance with the arguments 'ICA(method='infomax', fit_params=dict(extended=True))' (scikit-learn) or 'ICA(method='picard', fit_params=dict(ortho=False, extended=True))' (python-picard).
  ic_labels = label_components(eeg_data, ica, method="iclabel")


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Excluding ICA components: [0, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18]
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 12 ICA components
    Projecting back using 62 PCA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
398 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 398 events and 2001 original time points ...
0 bad epochs dropped
Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/62 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/62 [00:00<?,       ?it/s]

KeyboardInterrupt: 

![cleaned rsEEG](img/rsEEG/rsEEG_cleaned.png)

In [57]:
rsEEG_copy.save(os.path.join(".", "filtered", "rsEEG_cleaned.fif"))

Writing /home/tomasgalle/UGent/thesis/tms-research/filtered/rsEEG_cleaned.fif


/tmp/ipykernel_1454481/284752700.py:1: RuntimeWarning: This filename (/home/tomasgalle/UGent/thesis/tms-research/filtered/rsEEG_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  rsEEG_copy.save(os.path.join(".", "filtered", "rsEEG_cleaned.fif"))


Closing /home/tomasgalle/UGent/thesis/tms-research/filtered/rsEEG_cleaned.fif
[done]


## Automated cleaning on all files in a directory
Apply rsEEG or spTEP cleaning appropriately

In [ ]:
import concurrent.futures
import os


def process_file(file_path):
    raw = mne.io.read_raw_brainvision(file_path, preload=True)

    # Apply correct pipeline
    if "rsEEG" in file_path:
        # Apply the rsEEG pipeline
        print(f"Processing rsEEG file: {file_path}")
    elif "spTEP" in file_path:
        # Apply the spTEP pipeline
        print(f"Processing spTEP file: {file_path}")

DATASET_PATH = os.path.join(".", "dataset")

files = [
    os.path.join(DATASET_PATH, file)
    for file in os.listdir(DATASET_PATH)
    if file.endswith(".vhdr")
]

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_file, file) for file in files]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        pass